In [1]:
import pandas as pd 
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# DT 문제 1번

#### ※함수에 들어가 있는 변수나 flow는 본인이 바꾸셔도 가능하며 결과만 똑같이 나오면 됩니다!
#### ※hard코딩(이 데이터셋에만 적용되는 코딩방법) 말고 전체 데이터에 적용 가능하게 함수를 짜주셔야 합니다.

# Data Loading

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수에 대한 함수 만들기.

- Input은 Dataframe과 label이름으로 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다!

In [3]:
def get_gini(df, label):
    gini = 1 - (df[label].value_counts(normalize=True)**2).sum()
    return gini

In [4]:
get_gini(pd_data, 'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})
- Powerset은 모든 조합을 뱉어내게 됩니다. 이건 그냥 완성된걸 드릴게요.

In [5]:
from itertools import chain, combinations

def powerset(feature_class):
    listed_data = list(feature_class)
    chain_set = chain.from_iterable(combinations(feature_class, i) 
                                    for i in range(len(feature_class)+1))
    return [set_data for set_data in chain_set]

In [6]:
powerset(pd_data.age.unique())

[(),
 ('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior'),
 ('youth', 'middle_aged', 'senior')]

In [7]:
pd_data.age.unique()

array(['youth', 'middle_aged', 'senior'], dtype=object)

- 저 위에 부분 집합 중 우리가 원하는 집합(이진 분류)만 골라 내야하겠죠?
- 그 함수를 get_binary_split로 완성해주세요!
- 완성된 내용은 다음과 같이 나와야 합니다.

In [8]:
#binary로 쌍에서 중복을 제거하는 함수 추가
def unique_binary(result, values):
    unique_result = []
    for i, binary in enumerate(result):
        if not i == len(result)-1:
            for binary2 in result[i+1:]: #i번째 값과 i+a번째 를 합쳐서 비교.
                if sorted(list(binary) + list(binary2)) == sorted(list(values)):
                    unique_result.append(binary)
    
    return unique_result

In [9]:
def get_binary_split(df, attribute):
    attr_values = df[attribute].unique()
    chain_set = chain.from_iterable(combinations(attr_values, i) 
                                    for i in range(len(attr_values)+1))
    result = [set_data for set_data in chain_set
                 if not (len(set_data) == 0 or len(set_data) == len(attr_values))]
    '''
    result는 중복을 포함하는 binary 조합
    result 예시 :
    [(),
     ('youth',),
     ('middle_aged',),
     ('senior',),
     ('youth', 'middle_aged'),
     ('youth', 'senior'),
     ('middle_aged', 'senior')]
    '''
    #위에서 추가한 중복 제거 함수 적용
    unique_result = unique_binary(result, attr_values)
    return unique_result

In [10]:
get_binary_split(pd_data, "age")

[('youth',), ('middle_aged',), ('senior',)]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 함수를 사용하여 만들어주세요!
- DataFrame의 index를 사용하여 만들면 굉장히 편합니다..! 예시를 아래에서 보여드릴게요.
- 결과는 아래 아래 아래 줄과 같아야 합니다.

In [11]:
def get_attribute_gini_index(df, attribute, label):
    binary_split = get_binary_split(df, attribute) #
    result = {}
    for binary in binary_split:
        #isin 함수는 값의 포함여부를 토대로 T/F 반환
        df1 = df[df[attribute].isin(binary)] #binary에 해당하는 행만 추출
        gini1 = get_gini(df1, label) #binary에 해당하는 행에서 gini index 구하기
        df2 = df[~df[attribute].isin(binary)] #binary에 해당하지 않는 행만 추출
        gini2 = get_gini(df2, label) #binary에 해당하지 않는 행에서 ginidex 구하기
        
        result["_".join(binary)] = len(df1)/len(df)*gini1 + len(df2)/len(df)*gini2
        #해당하는 binary의 Gini(A) 구하기
    return result

In [12]:
#여러 바이너리 Gini index중 최소 바이너리 반환하는 함수 추가
def min_gini(df, attribute, label):
    dic = get_attribute_gini_index(df, attribute, label)
    
    return {k:v for k, v in dic.items() if v == min(dic.values())}

In [13]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713}

- 여기서 가장 작은값으로 분류를 해야겠죠?

In [14]:
min_gini(pd_data, "age", "class_buys_computer")

{'middle_aged': 0.35714285714285715}

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) income의 이진분류를 얻는 함수 get_binary_split(pd_data, "income")을 통해 보여주세요.

## 문제2) 가장 Gini계수가 낮은 Feature 즉 분류를 하는데 가장 중요한 변수를 선정하시고 get_attribute_gini_index함수를 통해 Gini index를 제시해주세요.

## 문제3) 2에서 구한 Feature로 DataFrame을 분류 해주시고 나눠진 2개의 클래스에서 각각 다음으로 중요한 Feature를 선정해주시고 Gini index를 제시해주세요.

In [15]:
#문제 1
get_binary_split(pd_data, "income")

[('high',), ('medium',), ('low',)]

In [16]:
# Gini 계수에 대한 정립 다시 필요.
print("age 변수의 최소 Gini 계수           : ", get_gini(pd_data, "age"))
print("income 변수의 최소 Gini 계수        : ", get_gini(pd_data, "income"))
print("student 변수의 최소 Gini 계수       : ", get_gini(pd_data, "student"))
print("credit_rating 변수의 최소 계수      : ", get_gini(pd_data, "credit_rating"))

#age 변수의 Gini 계수           :  0.6632653061224489
#income 변수의 Gini 계수        :  0.653061224489796
#student 변수의 Gini 계수       :  0.5
#credit_rating 변수의 Gini 계수 :  0.48979591836734704

#Gini 계수가 작을수록 정보가 명확하다는 의미

age 변수의 최소 Gini 계수           :  0.6632653061224489
income 변수의 최소 Gini 계수        :  0.653061224489796
student 변수의 최소 Gini 계수       :  0.5
credit_rating 변수의 최소 계수      :  0.48979591836734704


In [17]:
def feature_selection(df, label):
    #변수들만 골라오기
    attributes = list(df.columns)
    attributes.remove(label)
    
    for attribute in attributes:
        print(attribute, " 변수의 최소 Gini index : ", min_gini(df, attribute, label))

In [18]:
#문제 2

feature_selection(pd_data, 'class_buys_computer')
#age 변수에서 middle_aged, youth/senior binary 케이스가 0.357로 gini_index 가 가장 낮다.

age  변수의 최소 Gini index :  {'middle_aged': 0.35714285714285715}
income  변수의 최소 Gini index :  {'high': 0.4428571428571429}
student  변수의 최소 Gini index :  {'no': 0.3673469387755103}
credit_rating  변수의 최소 Gini index :  {'fair': 0.42857142857142855}


In [136]:
#문제 3
left_df = pd_data[pd_data["age"] == "middle_aged"]
right_df = pd_data[pd_data["age"] != "middle_aged"]
left_df
right_df

#feature_selection(left_df, 'class_buys_computer')
#left_df는 레이블이 yes 하나이므로 Gini index 0 값을 가진다.

feature_selection(right_df, 'class_buys_computer')
#right_df는 student 변수에 대해
# no, yes binary 케이스가 0.32로 gini_index가 가장 낮다. 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


age  변수의 최소 Gini index :  {'youth': 0.48}
income  변수의 최소 Gini index :  {'high': 0.375}
student  변수의 최소 Gini index :  {'no': 0.31999999999999984}
credit_rating  변수의 최소 Gini index :  {'fair': 0.4166666666666667}


In [93]:
#문제 2

print("age 변수의 최소 Gini index           : ", min_gini(pd_data, "age", "class_buys_computer"))
print("income 변수의 최소 Gini index        : ", min_gini(pd_data, "income", "class_buys_computer"))
print("student 변수의 최소 Gini index       : ", min_gini(pd_data, "student", "class_buys_computer"))
print("credit_rating 변수의 최소 Gini index : ", min_gini(pd_data, "credit_rating", "class_buys_computer"))

#age 변수에서 middle_aged, youth/senior binary 케이스가 0.357로 gini_index 가 가장 낮다.

age 변수의 최소 Gini index           :  {'middle_aged': 0.35714285714285715}
income 변수의 최소 Gini index        :  {'high': 0.4428571428571429}
student 변수의 최소 Gini index       :  {'no': 0.3673469387755103}
credit_rating 변수의 최소 Gini index :  {'fair': 0.42857142857142855}
